In [1]:
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser
from tqdm import tqdm

import random
import numpy as np
import pandas as pd

import pickle
import datetime as dt
import time

import itertools
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

import cv2

import keras

from keras.callbacks import TensorBoard

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

from mpl_toolkits.axes_grid1 import ImageGrid

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

from keras.models import Sequential
from keras.models import Model


from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.layers import Dense, GlobalAveragePooling2D
from keras import optimizers
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau


from keras import backend as K

from keras.applications import vgg16, resnet50, mobilenet
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications import xception
from keras.applications import inception_v3

from keras.applications.inception_v3 import InceptionV3

from keras.applications.vgg16 import preprocess_input, decode_predictions

from keras.optimizers import SGD




np.random.seed(2)
sns.set(style='white', context='notebook', palette='deep')

C:\Users\Andromeda\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def show_pred(preds, Y, val_breed, index, seq, ran):
    leng = len(preds)
    if seq:
        for i in range(index):
            if ran:
                index = random.randint(0, leng) 
            _, imagenet_class_name, prob = decode_predictions(preds, top=1)[index][0]
            plt.title("Original: " + val_breed[Y[index]] + "\nPrediction: " + imagenet_class_name)
            plt.imshow(X_train[index])
            plt.show()
    else:
            _, imagenet_class_name, prob = decode_predictions(preds, top=1)[index][0]
            plt.title("Original: " + val_breed[Y[index]] + "\nPrediction: " + imagenet_class_name)
            plt.imshow(X_train[index])
            plt.show()
        
def accuracy_func(preds, Y, val_breed):
    leng = len(preds)
    count = 0;
    for i in range(leng):
        _, imagenet_class_name, prob = decode_predictions(preds, top=1)[i][0]
        if val_breed[Y[i]] == imagenet_class_name:
            count+=1
    accuracy = (count/leng)*100
    
    print("Accuracy: ", accuracy)
    return accuracy

In [3]:
cache_dir = expanduser(join('~', '.keras'))
if not exists(cache_dir):
    makedirs(cache_dir)
models_dir = join(cache_dir, 'models')
if not exists(models_dir):
    makedirs(models_dir)

In [4]:
#Load the VGG model
vgg_model = vgg16.VGG16(weights='imagenet')
 
#Load the Inception_V3 model
inception_model = inception_v3.InceptionV3(weights='imagenet')
 
#Load the ResNet50 model
resnet_model = resnet50.ResNet50(weights='imagenet')
 
#Load the MobileNet model
mobilenet_model = mobilenet.MobileNet(weights='imagenet')

In [5]:
training_path = r'data/train'
validation_path = r'data/validation'
testing_path = r'data/test'
batch_size = 32
target_size=(224, 224)
norm = 255.0
class_mode='categorical'

In [6]:
train_datagen = ImageDataGenerator(
        rescale=1./norm,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./norm)
test_datagen = ImageDataGenerator(rescale=1./norm)

train_generator = train_datagen.flow_from_directory(
        training_path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode)

validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode)

test_generator = test_datagen.flow_from_directory(
        testing_path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode)

Found 3031 images belonging to 5 classes.
Found 866 images belonging to 5 classes.
Found 426 images belonging to 5 classes.


In [24]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

In [8]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(5, activation='softmax')(x)


In [9]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [10]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True



0 input_5
1 conv2d_95
2 batch_normalization_95
3 activation_144
4 conv2d_96
5 batch_normalization_96
6 activation_145
7 conv2d_97
8 batch_normalization_97
9 activation_146
10 max_pooling2d_6
11 conv2d_98
12 batch_normalization_98
13 activation_147
14 conv2d_99
15 batch_normalization_99
16 activation_148
17 max_pooling2d_7
18 conv2d_103
19 batch_normalization_103
20 activation_152
21 conv2d_101
22 conv2d_104
23 batch_normalization_101
24 batch_normalization_104
25 activation_150
26 activation_153
27 average_pooling2d_10
28 conv2d_100
29 conv2d_102
30 conv2d_105
31 conv2d_106
32 batch_normalization_100
33 batch_normalization_102
34 batch_normalization_105
35 batch_normalization_106
36 activation_149
37 activation_151
38 activation_154
39 activation_155
40 mixed0
41 conv2d_110
42 batch_normalization_110
43 activation_159
44 conv2d_108
45 conv2d_111
46 batch_normalization_108
47 batch_normalization_111
48 activation_157
49 activation_160
50 average_pooling2d_11
51 conv2d_107
52 conv2d_109


In [11]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate

model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=["accuracy"])

In [12]:
model_dir = 'output/models/'
log_file = "output/logs"

model_file = model_dir+"weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"

In [13]:
checkpoint = keras.callbacks.ModelCheckpoint(model_file, monitor='val_acc', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

In [14]:
tensorboard = keras.callbacks.TensorBoard(log_dir=log_file, histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None)
tensorboard.set_model(model) 

In [15]:
callbacks_list = [checkpoint, tensorboard]

In [16]:
# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history = model.fit_generator(
    train_generator,
    epochs=200,
    validation_data=validation_generator,
    verbose=1,
    callbacks=callbacks_list)

Epoch 1/100
95/95 [==============================] - ETA: 1:08:37 - loss: 1.8640 - acc: 0.03 - ETA: 43:03 - loss: 1.8273 - acc: 0.1250 - ETA: 35:03 - loss: 1.8006 - acc: 0.12 - ETA: 30:18 - loss: 1.7888 - acc: 0.14 - ETA: 28:05 - loss: 1.7822 - acc: 0.15 - ETA: 26:26 - loss: 1.7959 - acc: 0.14 - ETA: 24:58 - loss: 1.7882 - acc: 0.13 - ETA: 23:53 - loss: 1.7944 - acc: 0.13 - ETA: 22:56 - loss: 1.7888 - acc: 0.13 - ETA: 22:22 - loss: 1.7767 - acc: 0.14 - ETA: 21:51 - loss: 1.7711 - acc: 0.13 - ETA: 21:28 - loss: 1.7735 - acc: 0.14 - ETA: 20:59 - loss: 1.7685 - acc: 0.14 - ETA: 20:20 - loss: 1.7632 - acc: 0.14 - ETA: 19:56 - loss: 1.7558 - acc: 0.15 - ETA: 19:33 - loss: 1.7568 - acc: 0.15 - ETA: 19:11 - loss: 1.7481 - acc: 0.16 - ETA: 18:44 - loss: 1.7526 - acc: 0.16 - ETA: 18:19 - loss: 1.7505 - acc: 0.16 - ETA: 17:58 - loss: 1.7453 - acc: 0.16 - ETA: 17:44 - loss: 1.7393 - acc: 0.17 - ETA: 17:21 - loss: 1.7361 - acc: 0.17 - ETA: 17:01 - loss: 1.7326 - acc: 0.17 - ETA: 16:40 - loss: 1.71

95/95 [==============================] - ETA: 16:13 - loss: 1.0585 - acc: 0.87 - ETA: 16:11 - loss: 1.1163 - acc: 0.76 - ETA: 15:56 - loss: 1.0945 - acc: 0.76 - ETA: 16:44 - loss: 1.1294 - acc: 0.73 - ETA: 17:12 - loss: 1.1192 - acc: 0.71 - ETA: 17:00 - loss: 1.1203 - acc: 0.71 - ETA: 16:43 - loss: 1.1072 - acc: 0.71 - ETA: 16:38 - loss: 1.1152 - acc: 0.69 - ETA: 16:21 - loss: 1.1160 - acc: 0.69 - ETA: 16:04 - loss: 1.1105 - acc: 0.69 - ETA: 15:53 - loss: 1.1011 - acc: 0.69 - ETA: 15:53 - loss: 1.1077 - acc: 0.69 - ETA: 15:42 - loss: 1.1088 - acc: 0.68 - ETA: 15:32 - loss: 1.1084 - acc: 0.69 - ETA: 15:16 - loss: 1.1102 - acc: 0.69 - ETA: 15:08 - loss: 1.1135 - acc: 0.68 - ETA: 14:57 - loss: 1.1222 - acc: 0.68 - ETA: 14:43 - loss: 1.1204 - acc: 0.68 - ETA: 14:29 - loss: 1.1290 - acc: 0.66 - ETA: 14:17 - loss: 1.1288 - acc: 0.66 - ETA: 14:09 - loss: 1.1312 - acc: 0.66 - ETA: 13:58 - loss: 1.1320 - acc: 0.66 - ETA: 13:48 - loss: 1.1326 - acc: 0.66 - ETA: 13:37 - loss: 1.1348 - acc: 0.66 -

95/95 [==============================] - ETA: 17:17 - loss: 0.7396 - acc: 0.78 - ETA: 18:26 - loss: 0.8011 - acc: 0.75 - ETA: 17:58 - loss: 0.8329 - acc: 0.77 - ETA: 17:21 - loss: 0.8330 - acc: 0.75 - ETA: 16:47 - loss: 0.8565 - acc: 0.73 - ETA: 16:25 - loss: 0.8686 - acc: 0.72 - ETA: 16:06 - loss: 0.8595 - acc: 0.74 - ETA: 15:47 - loss: 0.8630 - acc: 0.75 - ETA: 15:32 - loss: 0.8696 - acc: 0.74 - ETA: 15:16 - loss: 0.8731 - acc: 0.74 - ETA: 15:02 - loss: 0.8658 - acc: 0.75 - ETA: 14:50 - loss: 0.8684 - acc: 0.74 - ETA: 14:35 - loss: 0.8670 - acc: 0.75 - ETA: 14:24 - loss: 0.8642 - acc: 0.75 - ETA: 14:12 - loss: 0.8525 - acc: 0.76 - ETA: 13:59 - loss: 0.8507 - acc: 0.76 - ETA: 13:47 - loss: 0.8440 - acc: 0.76 - ETA: 13:36 - loss: 0.8406 - acc: 0.76 - ETA: 13:24 - loss: 0.8355 - acc: 0.77 - ETA: 13:13 - loss: 0.8389 - acc: 0.76 - ETA: 13:02 - loss: 0.8371 - acc: 0.77 - ETA: 12:50 - loss: 0.8375 - acc: 0.76 - ETA: 12:39 - loss: 0.8356 - acc: 0.76 - ETA: 12:29 - loss: 0.8317 - acc: 0.77 -

95/95 [==============================] - ETA: 15:51 - loss: 0.6499 - acc: 0.71 - ETA: 15:40 - loss: 0.7009 - acc: 0.71 - ETA: 15:16 - loss: 0.6570 - acc: 0.77 - ETA: 15:08 - loss: 0.6627 - acc: 0.78 - ETA: 14:59 - loss: 0.6725 - acc: 0.78 - ETA: 14:45 - loss: 0.6645 - acc: 0.78 - ETA: 14:36 - loss: 0.6501 - acc: 0.80 - ETA: 14:24 - loss: 0.6519 - acc: 0.80 - ETA: 14:15 - loss: 0.6688 - acc: 0.79 - ETA: 14:05 - loss: 0.6735 - acc: 0.78 - ETA: 13:53 - loss: 0.6748 - acc: 0.78 - ETA: 13:44 - loss: 0.6717 - acc: 0.79 - ETA: 13:33 - loss: 0.6696 - acc: 0.80 - ETA: 13:23 - loss: 0.6799 - acc: 0.80 - ETA: 13:13 - loss: 0.6701 - acc: 0.80 - ETA: 13:02 - loss: 0.6678 - acc: 0.80 - ETA: 12:53 - loss: 0.6664 - acc: 0.80 - ETA: 12:44 - loss: 0.6719 - acc: 0.80 - ETA: 12:33 - loss: 0.6739 - acc: 0.80 - ETA: 12:24 - loss: 0.6747 - acc: 0.80 - ETA: 12:13 - loss: 0.6795 - acc: 0.79 - ETA: 12:03 - loss: 0.6804 - acc: 0.79 - ETA: 11:54 - loss: 0.6821 - acc: 0.79 - ETA: 11:43 - loss: 0.6862 - acc: 0.79 -

95/95 [==============================] - ETA: 18:31 - loss: 0.5730 - acc: 0.81 - ETA: 17:47 - loss: 0.5430 - acc: 0.82 - ETA: 17:34 - loss: 0.5413 - acc: 0.83 - ETA: 17:24 - loss: 0.5586 - acc: 0.82 - ETA: 17:09 - loss: 0.5576 - acc: 0.83 - ETA: 17:23 - loss: 0.5755 - acc: 0.81 - ETA: 17:12 - loss: 0.5944 - acc: 0.80 - ETA: 17:03 - loss: 0.6290 - acc: 0.78 - ETA: 17:03 - loss: 0.6157 - acc: 0.79 - ETA: 16:46 - loss: 0.6014 - acc: 0.80 - ETA: 16:37 - loss: 0.6127 - acc: 0.80 - ETA: 16:16 - loss: 0.5959 - acc: 0.80 - ETA: 15:53 - loss: 0.6009 - acc: 0.81 - ETA: 15:35 - loss: 0.5955 - acc: 0.81 - ETA: 15:17 - loss: 0.5951 - acc: 0.81 - ETA: 14:59 - loss: 0.5950 - acc: 0.81 - ETA: 14:43 - loss: 0.6012 - acc: 0.81 - ETA: 14:27 - loss: 0.5972 - acc: 0.81 - ETA: 14:11 - loss: 0.5974 - acc: 0.81 - ETA: 13:57 - loss: 0.5978 - acc: 0.81 - ETA: 13:43 - loss: 0.5971 - acc: 0.81 - ETA: 13:30 - loss: 0.6088 - acc: 0.80 - ETA: 13:17 - loss: 0.6094 - acc: 0.80 - ETA: 13:03 - loss: 0.6108 - acc: 0.80 -

94/95 [============================>.] - ETA: 16:14 - loss: 0.4273 - acc: 0.90 - ETA: 15:54 - loss: 0.4338 - acc: 0.89 - ETA: 15:35 - loss: 0.4990 - acc: 0.84 - ETA: 15:34 - loss: 0.4976 - acc: 0.85 - ETA: 15:21 - loss: 0.4919 - acc: 0.85 - ETA: 15:11 - loss: 0.5013 - acc: 0.84 - ETA: 15:03 - loss: 0.4848 - acc: 0.86 - ETA: 14:50 - loss: 0.4763 - acc: 0.86 - ETA: 14:41 - loss: 0.4682 - acc: 0.86 - ETA: 14:37 - loss: 0.4652 - acc: 0.86 - ETA: 14:28 - loss: 0.4879 - acc: 0.85 - ETA: 14:17 - loss: 0.4829 - acc: 0.86 - ETA: 14:07 - loss: 0.4851 - acc: 0.86 - ETA: 13:56 - loss: 0.4893 - acc: 0.86 - ETA: 13:49 - loss: 0.4941 - acc: 0.86 - ETA: 13:38 - loss: 0.4950 - acc: 0.86 - ETA: 13:27 - loss: 0.4870 - acc: 0.86 - ETA: 13:16 - loss: 0.4872 - acc: 0.85 - ETA: 13:06 - loss: 0.4867 - acc: 0.85 - ETA: 12:55 - loss: 0.4963 - acc: 0.85 - ETA: 12:45 - loss: 0.4906 - acc: 0.85 - ETA: 12:34 - loss: 0.4928 - acc: 0.85 - ETA: 12:23 - loss: 0.4926 - acc: 0.85 - ETA: 12:14 - loss: 0.5003 - acc: 0.85 -

KeyboardInterrupt: 

In [17]:
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [26]:
import h5py

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(5, activation='softmax')(x)


# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')




# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
# for i, layer in enumerate(base_model.layers):
#     print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

    

    
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate

model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=["accuracy"])






In [30]:
filename = 'output/models/weights-improvement-10-0.74.hdf5'
checkpoint = ModelCheckpoint(filepath=filename)

tensorboard = keras.callbacks.TensorBoard(log_dir=log_file, histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None)
tensorboard.set_model(model) 

callbacks_list = [checkpoint, tensorboard]


In [39]:
# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
preds = model.predict_generator(test_generator, verbose=1)

14/14 [==============================] - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 59s - ETA: 47 - ETA: 35 - ETA: 23 - ETA: 11 - 157s 11s/step


In [40]:
preds

array([[0.08001994, 0.5968215 , 0.15215993, 0.08518913, 0.08580942],
       [0.07988948, 0.15277135, 0.3950066 , 0.13173246, 0.24060015],
       [0.06730071, 0.08695392, 0.48542568, 0.20274799, 0.15757175],
       ...,
       [0.21741334, 0.29514855, 0.21954712, 0.15516518, 0.11272575],
       [0.0820367 , 0.36382902, 0.27622166, 0.08878279, 0.18912986],
       [0.08959825, 0.45473355, 0.19283439, 0.07145784, 0.19137593]],
      dtype=float32)

In [18]:
def get_class_values(gen):
    vals = []
    for i in gen:
        vals.append(gen[i])
    return vals

def get_classes(gen):
    rev_gen = {}
    for i in gen:
        rev_gen[gen[i]] = i
    return rev_gen

def find_closest (target, sets):
    num = -1
    diff = 10000
    for i in sets:
        if abs(i-target)<diff:
            num=i
            diff = abs(i-target)
    
    return num  

def get_label(cls, val):
    return cls[val]

In [19]:
vals = get_class_values(test_generator.class_indices)
cls = get_classes(test_generator.class_indices)
test_generator.class_indices

{'affenpinscher': 0,
 'afghan_hound': 1,
 'african_hunting_dog': 2,
 'airedale': 3,
 'american_staffordshire_terrier': 4,
 'appenzeller': 5,
 'australian_terrier': 6,
 'basenji': 7,
 'basset': 8,
 'beagle': 9,
 'bedlington_terrier': 10,
 'bernese_mountain_dog': 11,
 'black-and-tan_coonhound': 12,
 'blenheim_spaniel': 13,
 'bloodhound': 14,
 'bluetick': 15,
 'border_collie': 16,
 'border_terrier': 17,
 'borzoi': 18,
 'boston_bull': 19,
 'bouvier_des_flandres': 20,
 'boxer': 21,
 'brabancon_griffon': 22,
 'briard': 23,
 'brittany_spaniel': 24,
 'bull_mastiff': 25,
 'cairn': 26,
 'cardigan': 27,
 'chesapeake_bay_retriever': 28,
 'chihuahua': 29,
 'chow': 30,
 'clumber': 31,
 'cocker_spaniel': 32,
 'collie': 33,
 'curly-coated_retriever': 34,
 'dandie_dinmont': 35,
 'dhole': 36,
 'dingo': 37,
 'doberman': 38,
 'english_foxhound': 39,
 'english_setter': 40,
 'english_springer': 41,
 'entlebucher': 42,
 'eskimo_dog': 43,
 'flat-coated_retriever': 44,
 'french_bulldog': 45,
 'german_shepherd'

In [28]:
testing_path_new = "data\\pre\\test"
for i in range(len(preds)):
    label = get_label(cls, find_closest(preds[i].argmax(), vals))
    file_path = os.path.join(testing_path_new, test_generator.filenames[i])
    img = cv2.imread(file_path, 3)
    plt.imshow(img)
    plt.title(file_path+" --- "+ label)
    plt.show()

NameError: name 'preds' is not defined